In [3]:
import os
import sys
import inspect
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import STOPWORDS,WordCloud
from gensim import corpora
import numpy as np
import string

ModuleNotFoundError: No module named 'gensim'

In [26]:
processed = pd.read_csv('../data/processed_tweet_data.csv')
processed.head()

,Unnamed: 0,created_at,source,original_text,clean_text,polarity,subjectivity,lang,favorite_count,retweet_count,original_author,followers_count,friends_count,possibly_sensitive,hashtags,user_mentions,place
0,38,Fri Apr 22 22:17:05 +0000 2022,"<a href=""http://twitter.com/download/android"" ...",RT @NorthstarCharts: The 10-year yield is tell...,The 10-year yield is telling us that there's a...,0.16,0.540000,en,0.0,43.0,davideiacovozzi,18,55,NaN,"[{'text': 'gold', 'indices': [116, 121]}, {'te...",NorthstarCharts,NaN
1,39,Fri Apr 22 13:44:53 +0000 2022,"<a href=""http://twitter.com/download/android"" ...",RT @MichaelAArouet: German 10y mortgage rate w...,"German 10y mortgage rate went from 0,8% to 2,5...",0.15,0.175000,en,0.0,32.0,davideiacovozzi,18,55,NaN,[],MichaelAArouet,NaN
2,41,Fri Apr 22 06:10:34 +0000 2022,"<a href=""http://twitter.com/download/android"" ...",RT @goldseek: When? https://t.co/kO2FfHKaZg,When? https://t.co/kO2FfHKaZg,0.00,0.000000,en,0.0,26.0,davideiacovozzi,18,55,False,[],goldseek,NaN
3,42,Thu Apr 21 17:22:09 +0000 2022,"<a href=""http://twitter.com/download/android"" ...",RT @charliebilello: The 30-year mortgage rate ...,The 30-year mortgage rate in the US rises to 5...,0.00,0.066667,en,0.0,213.0,davideiacovozzi,18,55,NaN,[],charliebilello,NaN
4,43,Thu Apr 21 10:32:26 +0000 2022,"<a href=""http://twitter.com/download/android"" ...",RT @biancoresearch: Rates rise until something...,Rates rise until something breaks … is anythin...,-0.40,0.400000,en,0.0,417.0,davideiacovozzi,18,55,False,[],biancoresearch,NaN


In [27]:
processed_tweets.columns

Index(['Unnamed: 0', 'created_at', 'source', 'original_text', 'clean_text',
       'polarity', 'subjectivity', 'lang', 'favorite_count', 'retweet_count',
       'original_author', 'followers_count', 'friends_count',
       'possibly_sensitive', 'hashtags', 'user_mentions', 'place'],
      dtype='object')

In [8]:
processed_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16465 entries, 0 to 16464
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          16465 non-null  int64  
 1   created_at          16465 non-null  object 
 2   source              16465 non-null  object 
 3   original_text       16465 non-null  object 
 4   clean_text          8220 non-null   object 
 5   polarity            16465 non-null  float64
 6   subjectivity        16465 non-null  float64
 7   lang                16465 non-null  object 
 8   favorite_count      16465 non-null  float64
 9   retweet_count       16465 non-null  float64
 10  original_author     16465 non-null  object 
 11  followers_count     16465 non-null  int64  
 12  friends_count       16465 non-null  int64  
 13  possibly_sensitive  6206 non-null   object 
 14  hashtags            16465 non-null  object 
 15  user_mentions       12158 non-null  object 
 16  plac

In [28]:
processed_tweets.shape

(16465, 17)

In [29]:
print("missing values in each column is: \n{}".format(processed_tweets.isnull().sum()))
print("total number of missing value is: {}".format(processed_tweets.isnull().sum().sum()))
print("Columns with missing values are:{}".format(processed_tweets.columns[processed_tweets.isnull().any()]))

missing values in each column is: 
Unnamed: 0                0
created_at                0
source                    0
original_text             0
clean_text             8245
polarity                  0
subjectivity              0
lang                      0
favorite_count            0
retweet_count             0
original_author           0
followers_count           0
friends_count             0
possibly_sensitive    10259
hashtags                  0
user_mentions          4307
place                  5707
dtype: int64
total number of missing value is: 28518
Columns with missing values are:Index(['clean_text', 'possibly_sensitive', 'user_mentions', 'place'], dtype='object')


In [30]:
import os
c_directory = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
p_directory = os.path.dirname(c_directory)
sys.path.insert(0, p_directory) 

In [31]:
import sys
clean_tweet = pd.DataFrame(columns=['clean_text','polarity']) #create a new DataFrame(clean tweet ) with seris clean_txt and
clean_tweet['clean_text'] =  processed['clean_text'].to_list()   #polarity convert the series clean text and polarity to list
clean_tweet['polarity'] =  processed['polarity'].to_list()
clean_tweet.head()


,clean_text,polarity
0,The 10-year yield is telling us that there's a...,0.16
1,"German 10y mortgage rate went from 0,8% to 2,5...",0.15
2,When? https://t.co/kO2FfHKaZg,0.00
3,The 30-year mortgage rate in the US rises to 5...,0.00
4,Rates rise until something breaks … is anythin...,-0.40


In [34]:
new_data = pd.DataFrame(columns=['original_text','polarity','lang','hashtags'])

new_data['original_text'] =  processed['original_text'].to_list()
new_data['polarity'] =  processed['polarity'].to_list()
new_data['lang'] = processed['lang'].to_list()
new_data['hashtags'] = processed['hashtags'].to_list()

new_data.head()

,original_text,polarity,lang,hashtags
0,RT @NorthstarCharts: The 10-year yield is tell...,0.16,en,"[{'text': 'gold', 'indices': [116, 121]}, {'te..."
1,RT @MichaelAArouet: German 10y mortgage rate w...,0.15,en,[]
2,RT @goldseek: When? https://t.co/kO2FfHKaZg,0.00,en,[]
3,RT @charliebilello: The 30-year mortgage rate ...,0.00,en,[]
4,RT @biancoresearch: Rates rise until something...,-0.40,en,[]


In [41]:
clean_tweet.add_clean_text(new_data)
new_data

ModuleNotFoundError: No module named 'clean_tweets_dataframe'